In [1]:
import pandas as pd
#data_url="/content/drugsComTrain_raw.csv"
data_url="C:/Users/Jammula/Dropbox/My PC (DESKTOP-L6TP5LR)/Downloads/drugsComTrain_raw.csv"
dataset = pd.read_csv(data_url, delimiter=',')
dataset.shape
dataset.head()

,drugName,condition,review,rating
0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",1
1,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",1
2,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",0
3,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",1
4,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",1


In [2]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

stopwords = stopwords.words('english')
stopwords.remove('not')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jammula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
corpus=[]
# removing special characters and converting the whole data into lower case
for i in range(len(dataset)):
  review = re.sub('[^a-zA-Z]', ' ', dataset['review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [4]:
corpus

['side effect take combin bystol mg fish oil',
 'son halfway fourth week intuniv becam concern began last week start take highest dose two day could hardli get bed cranki slept nearli hour drive home school vacat unusu call doctor monday morn said stick day see school get morn last two day problem free much agreeabl ever less emot good thing less cranki rememb thing overal behavior better tri mani differ medic far effect',
 'use take anoth oral contracept pill cycl happi light period max day side effect contain hormon gestoden not avail us switch lybrel ingredi similar pill end start lybrel immedi first day period instruct said period last two week take second pack two week third pack thing got even wors third period last two week end third week still daili brown discharg posit side side effect idea period free tempt ala',
 'first time use form birth control glad went patch month first decreas libido subsid downsid made period longer day exact use period day max also made cramp intens 

In [5]:
%pip install scikit-learn
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1420)

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Jammula\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

In [7]:
import pickle
bowpath = 'BoW_Sentiment_Model.pkl'
pickle.dump(cv, open(bowpath, "wb"))

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [9]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [10]:
import joblib
joblib.dump(classifier, 'Classifier_Sentiment_Model')

['Classifier_Sentiment_Model']

In [11]:
y_pred = classifier.predict(X_test)

from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)

0.6455362678239306

In [12]:
import pandas as pd
data_url="C:/Users/Jammula/Dropbox/My PC (DESKTOP-L6TP5LR)/Downloads/drugsComTest.csv"
dataset = pd.read_csv(data_url, delimiter=',')
dataset.shape
dataset.head()

,drugName,condition,review
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th..."
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ..."
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms"""
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al..."
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc..."


In [13]:
import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

stopwords = stopwords.words('english')
stopwords.remove('not')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jammula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
corpus=[]

for i in range(len(dataset)):
  review = re.sub('[^a-zA-Z]', ' ', dataset['review'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in set(stopwords)]
  review = ' '.join(review)
  corpus.append(review)

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle
cvFile='BoW_Sentiment_Model.pkl'

cv = pickle.load(open(cvFile, "rb"))

In [16]:
X_fresh = cv.transform(corpus).toarray()
X_fresh.shape

(6355, 1420)

In [17]:
import joblib
classifier = joblib.load('Classifier_Sentiment_Model')

In [18]:
y_pred = classifier.predict(X_fresh)
print(y_pred)

[1 1 0 ... 1 0 0]


In [19]:
dataset['predicted_label'] = y_pred.tolist()
dataset.head(100)

,drugName,condition,review,predicted_label
0,Mirtazapine,Depression,"""I&#039;ve tried a few antidepressants over th...",1
1,Mesalamine,"Crohn's Disease, Maintenance","""My son has Crohn&#039;s disease and has done ...",1
2,Bactrim,Urinary Tract Infection,"""Quick reduction of symptoms""",0
3,Contrave,Weight Loss,"""Contrave combines drugs that were used for al...",1
4,Cyclafem 1 / 35,Birth Control,"""I have been on this birth control for one cyc...",1
...,...,...,...,...
95,Humira,Psoriatic Arthritis,"""I really like this medication! It helps me wi...",0
96,"Insulin inhalation, rapid acting","Diabetes, Type 1","""Horrible""",0
97,Acetaminophen / hydrocodone,Pain,"""I have been prescribed Vicodin 5/500s for ove...",1
98,Phenazopyridine,Interstitial Cystitis,"""I use Azo-Standard whenever I have issues wit...",1


In [32]:
import pandas as pd
import re
import nltk
import tkinter as tk
from tkinter import ttk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer  # Use TfidfVectorizer
import pickle
import joblib

# Load NLTK resources
nltk.download('stopwords')

# Load the pre-trained models
cvFile = 'BoW_Sentiment_Model.pkl'
cv = pickle.load(open(cvFile, "rb"))
classifier = joblib.load('Classifier_Sentiment_Model')

# Create a function to perform sentiment analysis
def perform_sentiment_analysis():
    review = text_input.get("1.0", "end-1c")
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
    review = ' '.join(review)
    review_vectorized = cv.transform([review]).toarray()  # Use CountVectorizer
    sentiment = classifier.predict(review_vectorized)[0]
    
    if sentiment == 0:
        result_label.config(text="Negative Sentiment")
    else:
        result_label.config(text="Positive Sentiment")

# Create a GUI window
window = tk.Tk()
window.title("Sentiment Analysis")

# Add a label and textbox
label = ttk.Label(window, text="Enter a review:")
label.pack()

text_input = tk.Text(window, height=5, width=40)
text_input.pack()

# Add a button to perform sentiment analysis
analyze_button = ttk.Button(window, text="Analyze", command=perform_sentiment_analysis)
analyze_button.pack()

# Add a label to display the result
result_label = ttk.Label(window, text="")
result_label.pack()

# Run the GUI event loop
window.mainloop()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jammula\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
